# Thai Wiki Language Model for Text Generation

This notebook details how you can use pretrained language model on [Thai Wikipedia Dump](https://dumps.wikimedia.org/thwiki/latest/thwiki-latest-pages-articles.xml.bz2) to generate texts.

In [1]:
# #uncomment if you are running from google colab
# !pip install sklearn_crfsuite
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
# !pip install fastai==1.0.45

In [3]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

#viz
import matplotlib.pyplot as plt
import seaborn as sns

#fastai
import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

#pythainlp
from pythainlp.ulmfit import *

In [4]:
#get dummy data
imdb = untar_data(URLs.IMDB_SAMPLE)
dummy_df = pd.read_csv(imdb/'texts.csv')

#get vocab
thwiki_itos = pickle.load(open(_THWIKI_LSTM['itos_fname'],'rb'))
thwiki_vocab = fastai.text.transform.Vocab(thwiki_itos)

#dummy databunch
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=thwiki_vocab, max_vocab=60000, min_freq=3)]
data_lm = (TextList.from_df(dummy_df, imdb, cols=['text'], processor=processor)
    .random_split_by_pct(0.2)
    .label_for_lm()
    .databunch(bs=64))


data_lm.sanity_check()

In [5]:
#check vocab size
len(data_lm.vocab.itos)

60005

In [6]:
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

#load pretrained models
learn.load_pretrained(**_THWIKI_LSTM)

In [7]:
learn.predict('กาลครั้งหนึ่งนานมาแล้ว ', 200, temperature=0.8, sep = '')

'กาลครั้งหนึ่งนานมาแล้ว พระโมคคัลลานะ ได้เสด็จไปยังพระวิหารบังนาค ณ พระราชวังสาพันปี ในป่าใกล้เมืองกุสินารา มีพระนาเจลเถระ ผู้เป็นพระโอรสของพระองค์ เป็นผู้มาโปรดให้มีการนมัสการ พร้อมเหล่าลบหลู่พระราชกุมาร เพื่อถวายศีล และถวายศีลแด่พระผู้มีพระภาค และพระเชษฐภคินีทั้ง 2 พระองค์ ประทับนั่งบนแท่นที่ประดิษฐาน ซึ่งอยู่เหนือเสา ซึ่งเป็นที่แสดงธรรมของพระศาสดาในอดีต \n \n เมื่อเลดี้ได้เดินทางไปยังบ้านของพระทำสัญญาทางศาสนา ได้เรียกกลุ่มของเหล่าสาวกและพระมหาเถระว่า "ไปหาที" ที่ตนสำเร็จเป็นพระอริยบุคคล และมาร่วมพิธีมิสซา และเล่มสุดท้ายในพระสูตร \n \n \n \n xxbos \n =เดอะบียอนเซ่ ในเพลง "ซัมเมอร์ฮาร์ตออฟร็อก= \n เดอะบลูดิสเวย์ '